In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
K.set_image_data_format('channels_last')
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
import PIL

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [16]:
#load the model and pre-trained weights
from tensorflow.keras.models import model_from_json
from inception_blocks_v2 import faceRecoModel
from tensorflow import keras

input_size = (3,160,160)
# json_file = open('/home/charchit/Desktop/DLS/face_recog/files/model.json','r')
# loaded_model_json = json_file.read()  
# json_file.close()
# model = model_from_json(loaded_model_json)
# model = faceRecoModel(input_size)
# model.load_weights('/home/charchit/Desktop/DLS/face_recog/files/model.h5')
model = keras.models.load_model('/home/charchit/Desktop/DLS/face_recog/files/model_data/')

In [18]:
model.inputs

[<KerasTensor: shape=(None, 160, 160, 3) dtype=float32 (created by layer 'input_1')>]

In [52]:
#model alias
FaceRecogModel = model


In [21]:
pwd

'/home/charchit/Desktop/DLS/face_recog'

In [22]:
aman_image_path = '/home/charchit/Desktop/DLS/face_recog/dataset/aman/aman8.jpg'
charchit_image_path = '/home/charchit/Desktop/DLS/face_recog/dataset/charchit/charchit0.png'
mummy_image_path = '/home/charchit/Desktop/DLS/face_recog/dataset/mummy/mummy6.jpg'
papa_image_path = '/home/charchit/Desktop/DLS/face_recog/dataset/papa/papa0.jpg'

In [30]:
#create the encoding of the images!
def img_to_encoding(image_path, model):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(160, 160))
    img = np.around(np.array(img) / 255.0, decimals=12)
    x_train = np.expand_dims(img, axis=0)
    embedding = model.predict_on_batch(x_train)
    return embedding / np.linalg.norm(embedding, ord=2)

In [49]:
a = img_to_encoding(mm2,FaceRecogModel)

In [31]:
#create the database of the users! key is the name and value is the encoding value
database = {}
database['charchit'] = img_to_encoding(charchit_image_path,FaceRecogModel)
database['aman'] = img_to_encoding(aman_image_path,FaceRecogModel)
database['papa'] = img_to_encoding(papa_image_path,FaceRecogModel)
database['mummy'] = img_to_encoding(mummy_image_path,FaceRecogModel)

In [32]:
database['charchit']

array([[ 0.01874726, -0.07499599, -0.0282793 , -0.10943334,  0.15293872,
         0.11324125, -0.06566025,  0.0615065 ,  0.06459101,  0.0169149 ,
         0.02558118,  0.00590753,  0.03023168, -0.14030972,  0.0472872 ,
        -0.06745273,  0.01753765, -0.06992961,  0.07642013, -0.08882043,
         0.01159177,  0.03147865, -0.07870072,  0.19004714, -0.02428116,
        -0.09496091,  0.01540461,  0.06299015, -0.01089316, -0.07820318,
         0.04826215, -0.02402242, -0.03399341, -0.06098218,  0.03467403,
        -0.06061645, -0.05250282, -0.00322592,  0.11566826,  0.03488366,
         0.00767145,  0.00624477,  0.09722745, -0.06317974, -0.17938863,
        -0.07759193, -0.01169309, -0.10483155,  0.05767546,  0.0104583 ,
        -0.07395659,  0.13654687,  0.15437137, -0.15690964,  0.00837531,
         0.10969198,  0.00831949, -0.12206151, -0.1214805 , -0.1017717 ,
        -0.06335504, -0.04564399, -0.016994  ,  0.14535485, -0.02845013,
         0.09542878,  0.09182978,  0.01603114, -0.0

In [36]:
#function to verify is the two person's are same!
def verify(image_path, identity, database, model):
    """
    Function that verifies if the person on the "image_path" image is "identity".
    
    Arguments:
        image_path -- path to an image
        identity -- string, name of the person you'd like to verify the identity. Has to be an employee who works in the office.
        database -- python dictionary mapping names of allowed people's names (strings) to their encodings (vectors).
        model -- your Inception model instance in Keras
    
    Returns:
        dist -- distance between the image_path and the image of "identity" in the database.
        door_open -- True, if the door should open. False otherwise.
    """

    # Step 1: Compute the encoding for the image. Use img_to_encoding() see example above. (≈ 1 line)
    encoding = img_to_encoding(image_path, model)
    # Step 2: Compute distance with identity's image (≈ 1 line)
    dist = np.linalg.norm(encoding - database[identity])
    # Step 3: Same person if dist < 0.7, else don't open (≈ 3 lines)
    if dist<0.7:
        print("It's " + str(identity) + ", Hey! welcome!")
        door_open = True
    else:
        print("It's not " + str(identity) + ", not the same person!")
        door_open = False
   
    return dist, door_open

In [48]:
mm2 = '/home/charchit/Desktop/DLS/face_recog/dataset/mummy/mummy8.jpg'

In [43]:
verify(mm2,'mummy',database,FaceRecogModel)

It's not mummy, not the same person!


(1.0290037, False)

In [56]:
def who_is_it(image_path, database, model):
    """
    Implements face recognition for the office by finding who is the person on the image_path image.
    
    Arguments:
        image_path -- path to an image
        database -- database containing image encodings along with the name of the person on the image
        model -- your Inception model instance in Keras
    
    Returns:
        min_dist -- the minimum distance between image_path encoding and the encodings from the database
        identity -- string, the name prediction for the person on image_path
    """
  
    ## Step 1: Compute the target "encoding" for the image. Use img_to_encoding() see example above. ## (≈ 1 line)
    encoding =  img_to_encoding(image_path,model)
    
    ## Step 2: Find the closest encoding ##
    
    # Initialize "min_dist" to a large value, say 100 (≈1 line)
    min_dist = 100
    
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        
        # Compute L2 distance between the target "encoding" and the current db_enc from the database. (≈ 1 line)
        dist = np.linalg.norm(encoding-db_enc)

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name. (≈ 3 lines)
        if dist<min_dist:
            min_dist = dist
            identity = name
    
    # if min_dist > 0.7:
    #     print("Not in the database.")
    # else:
    #     print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

In [57]:
who_is_it(mm2,database,FaceRecogModel)

(1.0290037, 'mummy')